In [6]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')
import tensorflow as tf

import math
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import pickle
import gzip
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        for i in range(len(gpus)):
            tf.config.experimental.set_memory_growth(gpus[i], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [7]:
import keras
keras.__version__

'2.4.3'

# Data Load

In [19]:
import random  
seed_num = 42
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((6182, 10, 4068), (6182,), (1545, 10, 4068), (1545,))

In [9]:
def MAE(y_true, y_pred):
    return "{:.2e}".format(np.mean(np.abs(y_true-y_pred)))

def MSE(y_true, y_pred):
    return "{:.2e}".format(np.mean(np.square(y_true-y_pred)))

def RMSE(y_true, y_pred):
    return "{:.2e}".format(np.sqrt(np.mean(np.square(y_true-y_pred))))

In [10]:
from sklearn.ensemble import AdaBoostRegressor, AdaBoostClassifier
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from sklearn.ensemble import VotingClassifier
from keras.layers import InputLayer, GRU

# Modeling

In [7]:
# # 2. LSTM model
# def lstm_model():
#     seed_num = 42 # model_seed
#     tf.random.set_seed(seed_num)

#     lstm = Sequential()
#     lstm.add(InputLayer(input_shape=(x.shape[1],x.shape[2])))
#     lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
#     lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
#     lstm.add(Dropout(0.2))
#     lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
#     lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
#     lstm.add(Dropout(0.2))
#     lstm.add(Dense(units=1, activation='sigmoid'))
#     return lstm

# def sorted_ls(path):
#     mtime=lambda f: os.stat(os.path.join(path, f)).st_mtime
#     return list(sorted(os.listdir(path), key=mtime))

# path = '/project/LSH/'
# with tf.device('/device:GPU:0'):
#     # 1. Data load
#     x = np.load(path + 'x_(7727,10,4068).npy')
#     y = np.load(path + 'y_(7727,1).npy')

#     # 3-1. Best model saving
#     MODEL_SAVE_FOLDER_PATH = './models/'
#     if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
#         os.mkdir(MODEL_SAVE_FOLDER_PATH)

#     acc_list, precision_list, recall_list, f1_list, auc_list = [], [], [], [], []
#     sss = StratifiedShuffleSplit(n_splits=10, test_size = 0.2, random_state = 42)

#     # 4. Crossvalidation
#     for seed, (train_index, test_index) in tqdm(enumerate(sss.split(x, y))):
#         X_train, y_train = x[train_index,:,:], y[train_index]
#         X_test, y_test = x[test_index,:,:], y[test_index]

#         # 3-2. Best model saving
#         model_path = MODEL_SAVE_FOLDER_PATH + f'/seed={seed}_'+'{epoch:02d}'+'_val_loss:{val_loss:.4f}.hdf5'

#         cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',
#                                         verbose=1, save_best_only=True)
#         early_stop = EarlyStopping(monitor='val_loss', patience=50, verbose=1, restore_best_weights=False)


#         model = lstm_model()
#         model.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), 
#                       loss = "binary_crossentropy", metrics=['acc'])
#         model.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=500,
#                   callbacks=[early_stop, cb_checkpoint], shuffle=False)


In [11]:
tf.random.set_seed(seed_num)

In [18]:
%%time
tf.random.set_seed(seed_num)
def get_model():
    lstm = Sequential()
    lstm.add(InputLayer(input_shape=(x.shape[1],x.shape[2])))
    lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(Dropout(0.2))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
    lstm.add(Dropout(0.2))
    lstm.add(Dense(units=1, activation='sigmoid'))

    lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), 
                          loss = "binary_crossentropy", metrics=['acc'])
    return lstm

estimator = []
for i in range(1,51):
    LSTM_Predictors = KerasClassifier(build_fn=lambda:get_model(), epochs=20, batch_size=256)
    LSTM_Predictors._estimator_type="classifier"
    estimator.append((f'model{i}', LSTM_Predictors))
print(estimator)    

# estimator = [('model1', LSTM_Predictors), ('model2', LSTM_Predictors1)]

final_model = VotingClassifier(estimators = estimator, voting = 'soft')
final_model.fit(X_train,y_train)

preds = final_model.predict(X_test)

preds[preds>0.5]=1
preds[preds<=0.5]=0

from sklearn import metrics 
print('정확도 :', metrics.accuracy_score(y_test, preds))
print("RMSE : ", RMSE(y_test, preds))

[('model1', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fa32029f850>), ('model2', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fa35c0a2af0>), ('model3', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fa35c0a2e20>), ('model4', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fa3187ed0d0>), ('model5', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fa3205f4430>), ('model6', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fa3205f4e80>), ('model7', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fa3205f4c40>), ('model8', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fa3205f4ee0>), ('model9', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fa3205f4d60>), ('model10', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object

KeyboardInterrupt: 

**KerasClassifier(build_fn=lambda:get_model(), epochs=20, batch_size=256)**
- range(1,2) : 0.747572
- range(1,11) : 0.7508090614886731
- range(1,6) : 0.7462783171521036
- range(1,21) : 0.7540453074433657
- 50개 : 0.7501618122977346

In [38]:
%%time
tf.random.set_seed(seed_num)
def get_model():
    lstm = Sequential()
    lstm.add(InputLayer(input_shape=(x.shape[1],x.shape[2])))
    lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(Dropout(0.2))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
    lstm.add(Dropout(0.2))
    lstm.add(Dense(units=1, activation='sigmoid'))

    lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), 
                          loss = "binary_crossentropy", metrics=['acc'])
    return lstm
with tf.device('/device:GPU:0'):
    estimator = []
    for i in range(1,51):
        LSTM_Predictors = KerasClassifier(build_fn=lambda:get_model(), epochs=100, batch_size=256)
        LSTM_Predictors._estimator_type="classifier"
        estimator.append((f'model{i}', LSTM_Predictors))
    print(estimator)    

    # estimator = [('model1', LSTM_Predictors), ('model2', LSTM_Predictors1)]

    final_model = VotingClassifier(estimators = estimator, voting = 'soft')
    final_model.fit(X_train,y_train)

    preds = final_model.predict(X_test)

    preds[preds>0.5]=1
    preds[preds<=0.5]=0

    from sklearn import metrics 
    print('정확도 :', metrics.accuracy_score(y_test, preds))
    print("RMSE : ", RMSE(y_test, preds))

[('model1', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fa320269460>), ('model2', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fa3e6ba08b0>), ('model3', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fa3e6ba0e80>), ('model4', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fa3e6ba0e50>), ('model5', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fa3e6ba0b50>), ('model6', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fa3e6ba0370>), ('model7', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fa3e6ba0fd0>), ('model8', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fa3e6ba0280>), ('model9', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fa3e6ba07f0>), ('model10', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object

Epoch 26/100
25/25 [==============================] - 2s 68ms/step - loss: 0.1367 - acc: 0.9609
Epoch 27/100
25/25 [==============================] - 2s 67ms/step - loss: 0.1405 - acc: 0.9615
Epoch 28/100
25/25 [==============================] - 2s 68ms/step - loss: 0.1270 - acc: 0.9655
Epoch 29/100
25/25 [==============================] - 2s 66ms/step - loss: 0.1275 - acc: 0.9646
Epoch 30/100
25/25 [==============================] - 2s 67ms/step - loss: 0.1244 - acc: 0.9660
Epoch 31/100
25/25 [==============================] - 2s 67ms/step - loss: 0.1240 - acc: 0.9665
Epoch 32/100
25/25 [==============================] - 2s 67ms/step - loss: 0.1240 - acc: 0.9662
Epoch 33/100
25/25 [==============================] - 2s 67ms/step - loss: 0.1185 - acc: 0.9681
Epoch 34/100
25/25 [==============================] - 2s 67ms/step - loss: 0.1118 - acc: 0.9717
Epoch 35/100
25/25 [==============================] - 2s 67ms/step - loss: 0.1040 - acc: 0.9749
Epoch 36/100
25/25 [====================

25/25 [==============================] - 2s 67ms/step - loss: 0.6082 - acc: 0.6679
Epoch 5/100
25/25 [==============================] - 2s 68ms/step - loss: 0.5103 - acc: 0.7676
Epoch 6/100
25/25 [==============================] - 2s 68ms/step - loss: 0.4395 - acc: 0.8067
Epoch 7/100
25/25 [==============================] - 2s 68ms/step - loss: 0.4152 - acc: 0.8261
Epoch 8/100
25/25 [==============================] - 2s 68ms/step - loss: 0.3716 - acc: 0.8499
Epoch 9/100
25/25 [==============================] - 2s 68ms/step - loss: 0.3447 - acc: 0.8682
Epoch 10/100
25/25 [==============================] - 2s 67ms/step - loss: 0.3248 - acc: 0.8792
Epoch 11/100
25/25 [==============================] - 2s 67ms/step - loss: 0.3097 - acc: 0.8885
Epoch 12/100
25/25 [==============================] - 2s 67ms/step - loss: 0.2866 - acc: 0.9029
Epoch 13/100
25/25 [==============================] - 2s 68ms/step - loss: 0.2676 - acc: 0.9097
Epoch 14/100
25/25 [==============================] - 2s 6

25/25 [==============================] - 2s 68ms/step - loss: 0.0876 - acc: 0.9795
Epoch 89/100
25/25 [==============================] - 2s 69ms/step - loss: 0.0808 - acc: 0.9825
Epoch 90/100
25/25 [==============================] - 2s 69ms/step - loss: 0.0793 - acc: 0.9832
Epoch 91/100
25/25 [==============================] - 2s 69ms/step - loss: 0.0793 - acc: 0.9838
Epoch 92/100
25/25 [==============================] - 2s 68ms/step - loss: 0.0899 - acc: 0.9782
Epoch 93/100
25/25 [==============================] - 2s 67ms/step - loss: 0.0797 - acc: 0.9837
Epoch 94/100
25/25 [==============================] - 2s 68ms/step - loss: 0.0786 - acc: 0.9832
Epoch 95/100
25/25 [==============================] - 2s 68ms/step - loss: 0.0776 - acc: 0.9837
Epoch 96/100
25/25 [==============================] - 2s 68ms/step - loss: 0.0762 - acc: 0.9837
Epoch 97/100
25/25 [==============================] - 2s 68ms/step - loss: 0.0777 - acc: 0.9837
Epoch 98/100
25/25 [==============================] -

25/25 [==============================] - 2s 67ms/step - loss: 0.1020 - acc: 0.9770
Epoch 67/100
25/25 [==============================] - 2s 69ms/step - loss: 0.1051 - acc: 0.9761
Epoch 68/100
25/25 [==============================] - 2s 68ms/step - loss: 0.1025 - acc: 0.9770
Epoch 69/100
25/25 [==============================] - 2s 68ms/step - loss: 0.0989 - acc: 0.9793
Epoch 70/100
25/25 [==============================] - 2s 68ms/step - loss: 0.0992 - acc: 0.9785
Epoch 71/100
25/25 [==============================] - 2s 68ms/step - loss: 0.1031 - acc: 0.9759
Epoch 72/100
25/25 [==============================] - 2s 69ms/step - loss: 0.1008 - acc: 0.9777
Epoch 73/100
25/25 [==============================] - 2s 67ms/step - loss: 0.1054 - acc: 0.9749
Epoch 74/100
25/25 [==============================] - 2s 67ms/step - loss: 0.0995 - acc: 0.9767
Epoch 75/100
25/25 [==============================] - 2s 68ms/step - loss: 0.0979 - acc: 0.9793
Epoch 76/100
25/25 [==============================] -

...

25/25 [==============================] - 2s 74ms/step - loss: 0.3028 - acc: 0.8971
Epoch 13/100
25/25 [==============================] - 2s 74ms/step - loss: 0.2802 - acc: 0.9086
Epoch 14/100
25/25 [==============================] - 2s 73ms/step - loss: 0.2760 - acc: 0.9080
Epoch 15/100
25/25 [==============================] - 2s 73ms/step - loss: 0.2544 - acc: 0.9196
Epoch 16/100
25/25 [==============================] - 2s 73ms/step - loss: 0.2546 - acc: 0.9160
Epoch 17/100
25/25 [==============================] - 2s 74ms/step - loss: 0.2430 - acc: 0.9266
Epoch 18/100
25/25 [==============================] - 2s 75ms/step - loss: 0.2400 - acc: 0.9245
Epoch 19/100
25/25 [==============================] - 2s 73ms/step - loss: 0.2143 - acc: 0.9377
Epoch 20/100
25/25 [==============================] - 2s 73ms/step - loss: 0.2160 - acc: 0.9368
Epoch 21/100
25/25 [==============================] - 2s 74ms/step - loss: 0.2023 - acc: 0.9452
Epoch 22/100
25/25 [==============================] -

25/25 [==============================] - 2s 73ms/step - loss: 0.0946 - acc: 0.9790
Epoch 98/100
25/25 [==============================] - 2s 73ms/step - loss: 0.0952 - acc: 0.9793
Epoch 99/100
25/25 [==============================] - 2s 73ms/step - loss: 0.0911 - acc: 0.9811
Epoch 100/100
25/25 [==============================] - 2s 72ms/step - loss: 0.0939 - acc: 0.9812
Epoch 1/100
25/25 [==============================] - 5s 74ms/step - loss: 0.6782 - acc: 0.5896
Epoch 2/100
25/25 [==============================] - 2s 75ms/step - loss: 0.6725 - acc: 0.6081
Epoch 3/100
25/25 [==============================] - 2s 75ms/step - loss: 0.6666 - acc: 0.6072
Epoch 4/100
25/25 [==============================] - 2s 72ms/step - loss: 0.6138 - acc: 0.6585
Epoch 5/100
25/25 [==============================] - 2s 74ms/step - loss: 0.5165 - acc: 0.7608
Epoch 6/100
25/25 [==============================] - 2s 75ms/step - loss: 0.4443 - acc: 0.8046
Epoch 7/100
25/25 [==============================] - 2s 73

25/25 [==============================] - 2s 71ms/step - loss: 0.0879 - acc: 0.9804
Epoch 76/100
25/25 [==============================] - 2s 72ms/step - loss: 0.0853 - acc: 0.9814
Epoch 77/100
25/25 [==============================] - 2s 72ms/step - loss: 0.0842 - acc: 0.9812
Epoch 78/100
25/25 [==============================] - 2s 71ms/step - loss: 0.0836 - acc: 0.9825
Epoch 79/100
25/25 [==============================] - 2s 72ms/step - loss: 0.0917 - acc: 0.9780
Epoch 80/100
25/25 [==============================] - 2s 71ms/step - loss: 0.0811 - acc: 0.9820
Epoch 81/100
25/25 [==============================] - 2s 71ms/step - loss: 0.0855 - acc: 0.9819
Epoch 82/100
25/25 [==============================] - 2s 73ms/step - loss: 0.0856 - acc: 0.9812
Epoch 83/100
25/25 [==============================] - 2s 72ms/step - loss: 0.0882 - acc: 0.9804
Epoch 84/100
25/25 [==============================] - 2s 72ms/step - loss: 0.0858 - acc: 0.9814
Epoch 85/100
25/25 [==============================] -

25/25 [==============================] - 2s 70ms/step - loss: 0.0927 - acc: 0.9801
Epoch 54/100
25/25 [==============================] - 2s 69ms/step - loss: 0.0912 - acc: 0.9796
Epoch 55/100
25/25 [==============================] - 2s 71ms/step - loss: 0.0904 - acc: 0.9809
Epoch 56/100
25/25 [==============================] - 2s 71ms/step - loss: 0.0874 - acc: 0.9804
Epoch 57/100
25/25 [==============================] - 2s 70ms/step - loss: 0.0936 - acc: 0.9762
Epoch 58/100
25/25 [==============================] - 2s 70ms/step - loss: 0.0914 - acc: 0.9795
Epoch 59/100
25/25 [==============================] - 2s 70ms/step - loss: 0.0931 - acc: 0.9782
Epoch 60/100
25/25 [==============================] - 2s 70ms/step - loss: 0.1081 - acc: 0.9730
Epoch 61/100
25/25 [==============================] - 2s 70ms/step - loss: 0.0922 - acc: 0.9778
Epoch 62/100
25/25 [==============================] - 2s 69ms/step - loss: 0.0932 - acc: 0.9780
Epoch 63/100
25/25 [==============================] -

25/25 [==============================] - 2s 72ms/step - loss: 0.1304 - acc: 0.9665
Epoch 32/100
25/25 [==============================] - 2s 73ms/step - loss: 0.1306 - acc: 0.9647
Epoch 33/100
25/25 [==============================] - 2s 72ms/step - loss: 0.1337 - acc: 0.9617
Epoch 34/100
25/25 [==============================] - 2s 72ms/step - loss: 0.1354 - acc: 0.9621
Epoch 35/100
25/25 [==============================] - 2s 72ms/step - loss: 0.1170 - acc: 0.9722
Epoch 36/100
25/25 [==============================] - 2s 74ms/step - loss: 0.1147 - acc: 0.9719
Epoch 37/100
25/25 [==============================] - 2s 73ms/step - loss: 0.1103 - acc: 0.9746
Epoch 38/100
25/25 [==============================] - 2s 73ms/step - loss: 0.1167 - acc: 0.9710
Epoch 39/100
25/25 [==============================] - 2s 73ms/step - loss: 0.1148 - acc: 0.9723
Epoch 40/100
25/25 [==============================] - 2s 72ms/step - loss: 0.1089 - acc: 0.9740
Epoch 41/100
25/25 [==============================] -

In [39]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
preds = final_model.predict(X_test)

precision = precision_score(y_test, preds)
recall = recall_score(y_test, preds)
f1 = f1_score(y_test, preds)
roc_auc = roc_auc_score(y_test, preds)
acc = accuracy_score(y_test, preds)

print(f' accuracy : {acc}, precision : {precision}, recall : {recall}, f1 : {f1}, roc_auc : {roc_auc}')

 accuracy : 0.7436893203883496, precision : 0.7903930131004366, recall : 0.7801724137931034, f1 : 0.7852494577006508, roc_auc : 0.7344946347733751


**KerasClassifier(build_fn=lambda:get_model(), epochs=100, batch_size=256)**
- 1개 : 0.7411003236245954
- 5개 : 0.7443365695792881
- 15개 : 0.7417475728155339
- 20개 : 0.7411003236245954
- 30개 : 
- 50개 : {accuracy : 0.7436893203883496, precision : 0.7903930131004366, recall : 0.7801724137931034, f1 : 0.7852494577006508, roc_auc : 0.7344946347733751}

In [32]:
%%time
tf.random.set_seed(seed_num)

def get_model(nums):
    lstm = Sequential()
    lstm.add(InputLayer(input_shape=(x.shape[1],x.shape[2])))
    lstm.add(LSTM(units=nums[0], activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=nums[1], activation='hard_sigmoid', return_sequences=True))
    lstm.add(Dropout(0.2))
    lstm.add(LSTM(units=nums[2], activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=nums[3], activation='hard_sigmoid', return_sequences=False))
    lstm.add(Dropout(0.2))
    lstm.add(Dense(units=1, activation='sigmoid'))

    lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), 
                          loss = "binary_crossentropy", metrics=['acc'])
    return lstm

# 서로 다른 모델을 만들어 합치기 (Model Ensemble)
model1 = KerasClassifier(build_fn=lambda:get_model(nums = [256, 128, 64, 32]), epochs=50, batch_size=256)
model1._estimator_type="classifier"
model2 = KerasClassifier(build_fn=lambda:get_model(nums = [128, 128, 64, 32]), epochs=50, batch_size=256)
model2._estimator_type="classifier"
model3 = KerasClassifier(build_fn=lambda:get_model(nums = [128, 64, 64, 32]), epochs=50, batch_size=256) 
model3._estimator_type="classifier"

estimator = [('model1', model1), ('model2', model2), ('model3', model3)]

final_model = VotingClassifier(estimators = estimator, voting = 'soft')
final_model.fit(X_train,y_train)

preds = final_model.predict(X_test)

preds[preds>0.5]=1
preds[preds<=0.5]=0

from sklearn import metrics 
print('정확도 :', metrics.accuracy_score(y_test, preds))
print("RMSE : ", RMSE(y_test, preds))

Epoch 1/50
25/25 [==============================] - 4s 67ms/step - loss: 0.6792 - acc: 0.5993
Epoch 2/50
25/25 [==============================] - 2s 66ms/step - loss: 0.6709 - acc: 0.6105
Epoch 3/50
25/25 [==============================] - 2s 66ms/step - loss: 0.6374 - acc: 0.6276
Epoch 4/50
25/25 [==============================] - 2s 67ms/step - loss: 0.5331 - acc: 0.7401
Epoch 5/50
25/25 [==============================] - 2s 66ms/step - loss: 0.4549 - acc: 0.8017
Epoch 6/50
25/25 [==============================] - 2s 67ms/step - loss: 0.4073 - acc: 0.8255
Epoch 7/50
25/25 [==============================] - 2s 66ms/step - loss: 0.3889 - acc: 0.8348
Epoch 8/50
25/25 [==============================] - 2s 67ms/step - loss: 0.3682 - acc: 0.8468
Epoch 9/50
25/25 [==============================] - 2s 66ms/step - loss: 0.3312 - acc: 0.8711
Epoch 10/50
25/25 [==============================] - 2s 66ms/step - loss: 0.3098 - acc: 0.8822
Epoch 11/50
25/25 [==============================] - 2s 66m

25/25 [==============================] - 2s 67ms/step - loss: 0.1752 - acc: 0.9489
Epoch 23/50
25/25 [==============================] - 2s 66ms/step - loss: 0.1659 - acc: 0.9523
Epoch 24/50
25/25 [==============================] - 2s 66ms/step - loss: 0.1548 - acc: 0.9588
Epoch 25/50
25/25 [==============================] - 2s 66ms/step - loss: 0.1439 - acc: 0.9604
Epoch 26/50
25/25 [==============================] - 2s 67ms/step - loss: 0.1470 - acc: 0.9594
Epoch 27/50
25/25 [==============================] - 2s 67ms/step - loss: 0.1367 - acc: 0.9636
Epoch 28/50
25/25 [==============================] - 2s 67ms/step - loss: 0.1426 - acc: 0.9621
Epoch 29/50
25/25 [==============================] - 2s 68ms/step - loss: 0.1324 - acc: 0.9651
Epoch 30/50
25/25 [==============================] - 2s 66ms/step - loss: 0.1294 - acc: 0.9655
Epoch 31/50
25/25 [==============================] - 2s 66ms/step - loss: 0.1275 - acc: 0.9672
Epoch 32/50
25/25 [==============================] - 2s 66ms/s

25/25 [==============================] - 2s 66ms/step - loss: 0.1150 - acc: 0.9743
정확도 : 0.7430420711974111
RMSE :  5.07e-01
CPU times: user 10min 34s, sys: 13min 40s, total: 24min 15s
Wall time: 4min 21s


In [ ]:
#https://machinelearningmastery.com/use-keras-deep-learning-models-scikit-learn-python/
from sklearn.model_selection import GridSearchCV

# Function to create model, required for KerasClassifier
def get_model(init='glorot_uniform'):
    lstm = Sequential()
    lstm.add(InputLayer(input_shape=(x.shape[1],x.shape[2])))
    lstm.add(LSTM(units=128, activation='hard_sigmoid', kernel_initializer=init, return_sequences=True))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', kernel_initializer=init, return_sequences=True))
    lstm.add(Dropout(0.2))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', kernel_initializer=init, return_sequences=True))
    lstm.add(LSTM(units=32, activation='hard_sigmoid', kernel_initializer=init, return_sequences=False))
    lstm.add(Dropout(0.2))
    lstm.add(Dense(units=1, activation='sigmoid'))

    lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), 
                          loss = "binary_crossentropy", metrics=['acc'])
    return lstm

# fix random seed for reproducibility
seed_num = 42
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# grid search epochs, batch size and optimizer
init = ['glorot_uniform', 'normal', 'uniform']
epochs = [50, 100, 150]
batches = [8, 16, 32, 64]
param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=init)

grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))